In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import numpy as np
import pandas as pd

import warnings
from pathlib import Path

from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold

import xgboost as xgb

In [ ]:
pd.set_option('max_columns', 100)
pd.set_option('display.precision', 4)

warnings.filterwarnings('ignore')

# 데이터 로드

In [ ]:
# 로컬 버전

# data_dir = Path('../input/dankook')
# feature_dir = Path('../output/feature')
# val_dir = Path('../output/oof_pred')
# test_dir = Path('../output/test_pred')
# sub_dir = Path('../output/sub')


# train_file = data_dir / 'train.csv'
# test_file = data_dir / 'test.csv'
# sample_file = data_dir / 'sample_submission.csv'

In [ ]:
# 코렙 

from google.colab import drive
drive.mount('/content/drive')

data_dir = Path('/content/drive/My Drive/Colab Notebooks/input/dankook')
feature_dir = Path('/content/drive/My Drive/Colab Notebooks/output/feature')
val_dir = Path('/content/drive/My Drive/Colab Notebooks/output/oof_pred')
test_dir = Path('/content/drive/My Drive/Colab Notebooks/output/test_pred')
sub_dir = Path('/content/drive/My Drive/Colab Notebooks/output/sub')

train_file = data_dir / 'train.csv'
test_file = data_dir / 'test.csv'
sample_file = data_dir / 'sample_submission.csv'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
algorithm_name = 'xgbcv'
feature_name = 'feature'
model_name = f'{algorithm_name}_{feature_name}'

feature_Ver1_file = feature_dir / f'{feature_name}_Ver1.csv'
feature_Ver2_file = feature_dir / f'{feature_name}_Ver2.csv'
feature_Ver3_file = feature_dir / f'{feature_name}_Ver3.csv'

xgb_oof_pred_ver1_file = val_dir / f'{model_name}_oof_pred_ver1.csv'
xgb_oof_pred_ver2_file = val_dir / f'{model_name}_oof_pred_ver2.csv'
xgb_oof_pred_ver3_file = val_dir / f'{model_name}_oof_pred_ver3.csv'

xgb_test_pred_ver1_file = test_dir / f'{model_name}_test_pred_ver1.csv'
xgb_test_pred_ver2_file = test_dir / f'{model_name}_test_pred_ver2.csv'
xgb_test_pred_ver3_file = test_dir / f'{model_name}_test_pred_ver3.csv'

xgb_submission_ver1_file = sub_dir / f'{model_name}_submission_Ver1.csv'
xgb_submission_ver2_file = sub_dir / f'{model_name}_submission_Ver2.csv'
xgb_submission_ver3_file = sub_dir / f'{model_name}_submission_Ver3.csv'

In [ ]:
SEED = 2020
num_class = 3
n_splits = 5
target_column = 'class'

## Ver1 데이터 로드

In [ ]:
dataset = pd.read_csv(feature_Ver1_file, index_col=0)
print(dataset.shape)
dataset.head()

(399923, 32)


,g,r,i,redshift,dered_u,dered_g,dered_r,dered_i,dered_z,nObserve,airmass_u,airmass_i,class,diff_g_u,diff_r_g,diff_i_r,diff_z_i,diff_dered_u_z,diff_dered_g_dered_u,diff_dered_r_dered_g,diff_dered_i_dered_r,diff_dered_z_dered_i,diff_airmass_g_airmass_u,u_0,u_1,g_1,r_0,r_1,i_0,i_1,z_0,z_1
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,20.3368,19.0095,17.6724,-8.1086e-05,23.1243,20.2578,18.9551,17.6321,16.9089,2.9444,1.1898,1.1894,0.0,-2.9272,-1.3273,-1.3371,-0.7328,6.1847,-2.8665,-1.3027,-1.3230,-0.7232,0.0009,0.1397,0.0790,0.0403,0.0307,22.0742,19.1461,17.8205,16.4831,15.7494
1,14.0620,13.4524,13.2684,4.5061e-03,14.9664,14.0045,13.4114,13.2363,13.1347,0.6931,1.2533,1.2510,1.0,-0.9902,-0.6096,-0.1840,-0.0995,1.7975,-0.9619,-0.5931,-0.1751,-0.1016,0.0046,0.0857,0.0574,0.0322,0.0343,13.7989,12.8041,12.2037,12.0175,11.9134
2,15.8254,15.5363,15.3935,4.7198e-04,16.6076,15.6866,15.4400,15.3217,15.2961,1.0986,1.0225,1.0217,0.0,-0.9610,-0.2891,-0.1428,-0.0435,1.2576,-0.9211,-0.2466,-0.1183,-0.0257,0.0016,0.1787,0.1388,0.0718,0.0540,15.7639,14.8013,14.5153,14.3718,14.3267
3,21.1887,20.2212,19.8949,5.8143e-06,25.3536,20.9947,20.0873,19.7947,19.5552,1.6094,1.2054,1.2051,0.0,-4.4719,-0.9676,-0.3262,-0.2603,5.7190,-4.3590,-0.9074,-0.2926,-0.2395,0.0007,0.3070,0.1941,0.1003,0.0795,24.4552,19.9827,19.0163,18.6898,18.4289
4,20.6992,19.0424,18.3242,-3.3247e-05,23.7714,20.4338,18.8630,18.1903,17.8759,2.6391,1.1939,1.1938,0.0,-3.7543,-1.6568,-0.7182,-0.3415,5.7888,-3.3376,-1.5709,-0.6727,-0.3144,0.0003,0.6820,0.2653,0.1339,0.1067,23.2595,19.5049,17.8486,17.1303,16.7885


In [ ]:
# train set
Ver1_X = dataset.loc[dataset[target_column] != -1 , :]
Ver1_X.drop(columns=target_column,inplace=True,axis=1)
Ver1_y = dataset.loc[dataset[target_column] != -1, target_column]
Ver1_y.astype(int)

# test set
Ver1_test = dataset.loc[dataset[target_column] == -1, :]
Ver1_test.drop(columns=target_column, inplace=True,axis=1)

print(Ver1_X.shape, Ver1_y.shape, Ver1_test.shape)

(319923, 31) (319923,) (80000, 31)


## Ver2 데이터 로드

In [ ]:
dataset = pd.read_csv(feature_Ver2_file, index_col=0)
print(dataset.shape)
dataset.head()

(399923, 42)


,redshift,dered_g,dered_r,dered_i,airmass_g,ugriz_r,ugriz_i,airmass_diff,ugriz_z_ugriz_g_diff,ugriz_z_ugriz_u_diff,ugriz_z_dered_z_diff,ugriz_z_dered_g_diff,ugriz_i_ugriz_r_diff,ugriz_i_dered_g_diff,ugriz_r_ugriz_g_diff,ugriz_r_ugriz_u_diff,ugriz_r_dered_z_diff,ugriz_r_dered_r_diff,ugriz_r_dered_g_diff,ugriz_r_dered_u_diff,ugriz_g_ugriz_u_diff,ugriz_g_dered_g_diff,ugriz_g_dered_u_diff,ugriz_u_dered_r_diff,ugriz_u_dered_g_diff,dered_z_dered_i_diff,dered_z_airmass_g_diff,dered_r_dered_g_diff,dered_r_dered_u_diff,dered_g_dered_u_diff,dered_g_airmass_i_diff,dered_g_airmass_g_diff,dered_u_airmass_i_diff,airmass_z_airmass_g_diff,ugriz_dered_min_diff,dered_airmass_sum_diff,ugriz_icolor,ugriz_ucolor,ugriz_p1,dered_orthogonal,dered_parallel,class
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,-8.1086e-05,20.2578,18.9551,17.6321,1.1907,19.0095,17.6724,0.0017,-3.3972,-6.3243,0.0307,-3.3182,-1.3371,-2.5854,-1.3273,-4.2545,2.1006,0.0544,-1.2483,-4.1148,-2.9272,0.0790,-2.7875,4.3088,3.0062,-0.7232,15.7183,-1.3027,-4.1691,-2.8665,19.0684,19.0671,21.9349,-0.0005,0.0307,90.9292,0.6094,0.0384,1.9346,0.8173,2.2835,0.0
1,4.5061e-03,14.0045,13.4114,13.2363,1.2578,13.4524,13.2684,0.0091,-0.8930,-1.8832,0.0343,-0.8356,-0.1840,-0.7361,-0.6096,-1.5997,0.3177,0.0410,-0.5521,-1.5140,-0.9902,0.0574,-0.9045,1.6408,1.0476,-0.1016,11.8768,-0.5931,-1.5550,-0.9619,12.7535,12.7467,13.7154,-0.0023,0.0343,62.4869,0.2947,0.1851,-0.1260,-0.1531,0.4094,1.0
2,4.7198e-04,15.6866,15.4400,15.3217,1.0241,15.5363,15.3935,0.0031,-0.4754,-1.4364,0.0540,-0.3366,-0.1428,-0.2931,-0.2891,-1.2501,0.2402,0.0963,-0.1503,-1.0713,-0.9610,0.1388,-0.7822,1.3463,1.0998,-0.0257,14.2720,-0.2466,-1.1676,-0.9211,14.6649,14.6625,15.5859,-0.0008,0.0540,73.2395,0.0617,-0.0031,-0.2855,-0.1233,0.0986,0.0
3,5.8143e-06,20.9947,20.0873,19.7947,1.2061,20.2212,19.8949,0.0012,-1.5541,-6.0260,0.0795,-1.3600,-0.3262,-1.0997,-0.9676,-5.4395,0.6660,0.1339,-0.7735,-5.1325,-4.4719,0.1941,-4.1649,5.5734,4.6660,-0.2395,18.3491,-0.9074,-5.2664,-4.3590,19.7895,19.7886,24.1485,-0.0003,0.0795,99.7582,-0.8936,-0.5544,3.1910,-0.1142,0.7703,0.0
4,-3.3247e-05,20.4338,18.8630,18.1903,1.1943,19.0424,18.3242,0.0005,-2.7165,-6.4708,0.1067,-2.4512,-0.7182,-2.1097,-1.6568,-5.4111,1.1664,0.1794,-1.3915,-4.7290,-3.7543,0.2653,-3.0722,5.5904,4.0196,-0.3144,16.6816,-1.5709,-4.9084,-3.3376,19.2400,19.2396,22.5776,-0.0002,0.1067,93.1645,0.1220,0.0117,2.8239,0.1000,1.6908,0.0


In [ ]:
# train set
Ver2_X = dataset.loc[dataset[target_column] != -1 , :]
Ver2_X.drop(columns=target_column,inplace=True,axis=1)
Ver2_y = dataset.loc[dataset[target_column] != -1, target_column]
Ver2_y.astype(int)

# test set
Ver2_test = dataset.loc[dataset[target_column] == -1, :]
Ver2_test.drop(columns=target_column, inplace=True,axis=1)

print(Ver2_X.shape, Ver2_y.shape, Ver2_test.shape)

(319923, 41) (319923,) (80000, 41)


## Ver 3 데이터 로드

In [ ]:
dataset = pd.read_csv(feature_Ver3_file, index_col=0)
print(dataset.shape)
dataset.head()

In [ ]:
# train set
Ver3_X = dataset.loc[dataset[target_column] != -1 , :]
Ver3_X.drop(columns=target_column,inplace=True,axis=1)
Ver3_y = dataset.loc[dataset[target_column] != -1, target_column]
Ver3_y.astype(int)

# test set
Ver3_test = dataset.loc[dataset[target_column] == -1, :]
Ver3_test.drop(columns=target_column, inplace=True,axis=1)

print(Ver3_X.shape, Ver3_y.shape, Ver3_test.shape)

# 모델 학습 

## Xgboost 

In [ ]:
# Xgboost
xgb_params = {
    'n_jobs' : -1, 
    'n_estimators': 100,
    'eval_metric': 'mlogloss',
    'eta': 0.3, # learning_rate
    'booster': 'gbtree',
    'objective': 'multi:softprob',
    'num_class': 3,
    'random_state': 2020,
    'tree_method': 'gpu_hist',
    'predictor': 'gpu_predictor'
}

In [ ]:
mlogloss = []

xgb_oof_pred_ver1 = np.zeros((Ver1_X.shape[0],num_class))
xgb_test_pred_ver1 = np.zeros((Ver1_test.shape[0],num_class))
xgb_oof_pred_ver2 = np.zeros((Ver2_X.shape[0],num_class))
xgb_test_pred_ver2 = np.zeros((Ver2_test.shape[0],num_class))
xgb_oof_pred_ver3 = np.zeros((Ver3_X.shape[0],num_class))
xgb_test_pred_ver3 = np.zeros((Ver3_test.shape[0],num_class))

y = Ver1_y


for X, test in [(Ver1_X,Ver1_test), (Ver2_X,Ver2_test), (Ver3_X, Ver3_test)]:
    kFold = StratifiedKFold(n_splits=n_splits, random_state=SEED, shuffle=True)
    for fold, (trn_idx, val_idx) in enumerate(kFold.split(X,y)):
        X_train, X_val = X.iloc[trn_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[trn_idx], y.iloc[val_idx]
        
        dtrain = xgb.DMatrix(X_train, label=y_train)
        dval = xgb.DMatrix(X_val, label=y_val)
        watchlist = [(dtrain, 'train'), (dval, 'val')]
        
        
        xgb_clf = xgb.train(params=xgb_params, dtrain=dtrain, num_boost_round=5000, evals=watchlist, early_stopping_rounds=50, verbose_eval= 5000)
        mlogloss.append(xgb_clf.best_score)
        
        # Predict 
        dtest = xgb.DMatrix(test)
        if X.shape[1]==31:
            xgb_test_pred_ver1 += xgb_clf.predict(dtest) / (n_splits*4)
            xgb_oof_pred_ver1[val_idx] += xgb_clf.predict(dval) / 4
        elif X.shape[1]==41:
            xgb_test_pred_ver2 += xgb_clf.predict(dtest) / (n_splits*4)
            xgb_oof_pred_ver2[val_idx] += xgb_clf.predict(dval) / 4
        else:
            xgb_test_pred_ver3 += xgb_clf.predict(dtest) / (n_splits*4)
            xgb_oof_pred_ver3[val_idx] += xgb_clf.predict(dval) / 4
        
    print("*"*100)
    print("Traning has finished")
print('xgb ver1 logloss= ',log_loss(y,xgb_oof_pred_ver1))
print('xgb ver2 logloss= ',log_loss(y,xgb_oof_pred_ver2))
print('xgb ver3 logloss= ',log_loos(y,xgb_oof_pred_ver3))

[0]	train-mlogloss:0.773226	val-mlogloss:0.774468
Multiple eval metrics have been passed: 'val-mlogloss' will be used for early stopping.

Will train until val-mlogloss hasn't improved in 50 rounds.
Stopping. Best iteration:
[76]	train-mlogloss:0.131925	val-mlogloss:0.158784

[0]	train-mlogloss:0.773525	val-mlogloss:0.773403
Multiple eval metrics have been passed: 'val-mlogloss' will be used for early stopping.

Will train until val-mlogloss hasn't improved in 50 rounds.
Stopping. Best iteration:
[61]	train-mlogloss:0.136326	val-mlogloss:0.156632

[0]	train-mlogloss:0.77306	val-mlogloss:0.773517
Multiple eval metrics have been passed: 'val-mlogloss' will be used for early stopping.

Will train until val-mlogloss hasn't improved in 50 rounds.
Stopping. Best iteration:
[89]	train-mlogloss:0.128663	val-mlogloss:0.157404

[0]	train-mlogloss:0.773111	val-mlogloss:0.773307
Multiple eval metrics have been passed: 'val-mlogloss' will be used for early stopping.

Will train until val-mlogloss h

# 제출 파일 및 기타 파일 생성

In [ ]:
# submission 파일 생성

sub = pd.read_csv(sample_file,index_col=0)

# Ver1 
sub[target_column] = np.argmax(xgb_test_pred_ver1, axis=1)
sub.to_csv(xgb_submission_ver1_file)

# Ver2
sub[target_column] = np.argmax(xgb_test_pred_ver2, axis=1)
sub.to_csv(xgb_submission_ver2_file)

# Ver3
sub[target_column] = np.argmax(xgb_test_pred_ver3, axis=1)
sub.to_csv(xgb_submission_ver3_file)

In [ ]:
# xgb_oof_pred 파일 생성

# Ver1
np.savetxt(xgb_oof_pred_ver1_file, xgb_oof_pred_ver1,fmt='%.18f',delimiter=',')

# Ver2
np.savetxt(xgb_oof_pred_ver2_file, xgb_oof_pred_ver2,fmt='%.18f', delimiter=',')

# Ver3
np.savetxt(xgb_oof_pred_ver3_file, xgb_oof_pred_ver3,fmt='%.18f', delimiter=',')

In [ ]:
# xgb_test_pred 파일 생성

# Ver1
np.savetxt(xgb_test_pred_ver1_file, xgb_test_pred_ver1,fmt='%.18f', delimiter=',')

# Ver2
np.savetxt(xgb_test_pred_ver2_file, xgb_test_pred_ver2,fmt='%.18f', delimiter=',')

# Ver3
np.savetxt(xgb_test_pred_ver3_file, xgb_test_pred_ver3,fmt='%.18f', delimiter=',')